# Acquisition de données avec une RaspberryPi

## Import librairies

In [ ]:
import requests, json, time
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import mysql.connector as cnt

## Fonctions de connection et d'éxecution

In [ ]:
def connect_db(host, user, pswd, database = None):
    db = cnt.connect(
        host = host,
        user = user,
        passwd = pswd,
        database = database
    )
    return(db)

def exe(db, request, value = None, verbose = False, name = False):
    cursor = db.cursor(buffered = True)
    
    if value:
        cursor.execute(request, value)

    else :
        cursor.execute(request)

    db.commit()
    
    if verbose:
        result = cursor.fetchall()
        if name:
            result.insert(0, cursor.column_names)
        return(result)

## Création d'une table pour insertion

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu')

queries = ['CREATE DATABASE IF NOT EXISTS RaspberryPi;']

queries.append("""
    CREATE TABLE IF NOT EXISTS RaspberryPi.Capteurs (
        Date timestamp PRIMARY KEY,
        Movemt BOOLEAN,
        Temp FLOAT,
        Humid FLOAT,
        Dist INT
    )ENGINE = InnoDB;    
""")

for query in queries:
    exe(db, query)

![Image](Images/Structure.jpg)

In [ ]:
json.loads(requests.get('http://192.168.20.27/modulesGrove.php?all').text)

## Récupération et insertion des données

In [ ]:
for i in range(5):
    try:
        db = connect_db('192.168.20.118', 'grogu', 'grogu')

        data = json.loads(requests.get('http://192.168.20.27/modulesGrove.php?all').text)
        query = (f"""
            INSERT INTO RaspberryPi.Capteurs (Movemt, Temp, Humid, Dist) VALUES (
                                                                                {data['PIR'] if data['PIR'] != '?' else 'NULL'}, 
                                                                                {data['temperature'] if data['temperature'] != '?' else 'NULL'}, 
                                                                                {data['humidite'] if data['humidite'] != '?' else 'NULL'}, 
                                                                                {data['US'] if data['US'] < 505 else 'NULL'});
        """)

        exe(db, query)

        time.sleep(1)

    except Exception as e:
        print(e)

![Image](Images/Resultat.jpg)

## Analyse des données graphiquement

In [ ]:
db = connect_db('192.168.20.118', 'grogu', 'grogu')
data = exe(db, 'SELECT * FROM RaspberryPi.Capteurs', verbose = True, name = True)

df = pd.DataFrame(data[1:], columns = data[0])
df['Date'] = df['Date'] + pd.Timedelta(hours=1)

fig, axes = plt.subplots(2, 2, figsize = (15, 15))
for i, column in enumerate(df.columns[1:]):
    if i < 2:
        axes[0][i].plot(df['Date'], df[column], drawstyle = "steps-mid")
        axes[0][i].set_title(column)
        axes[0][i].tick_params(labelrotation=45)
    else:
        axes[1][i-2].plot(df['Date'], df[column], drawstyle = "steps-mid")
        axes[1][i-2].set_title(column)
        axes[1][i-2].tick_params(labelrotation=45)

plt.show()

![Image](Images/Courbes.jpg)

## Graphique en temps réel

In [ ]:
def get_data():
    db = connect_db('192.168.20.118', 'grogu', 'grogu')
    data = exe(db, 'SELECT * FROM RaspberryPi.Capteurs', verbose = True, name = True)
    df = pd.DataFrame(data[1:], columns = data[0])
    df['Date'] = df['Date'] + pd.Timedelta(hours=1)
    return(df)

def draw_plot():
    df = get_data()


fig, axes = plt.subplots(2, 2, figsize = (15, 15))

df = get_data()

for i, column in enumerate(df.columns[1:]):
    if i < 2:
        axes[0][i].plot(df['Date'], df[column])
        axes[0][i].set_title(column)
        axes[0][i].tick_params(labelrotation=45)
    else:
        axes[1][i-2].plot(df['Date'], df[column])
        axes[1][i-2].set_title(column)
        axes[1][i-2].tick_params(labelrotation=45)

ani = animation.FuncAnimation(plt.gcf(), draw_plot, interval=20)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd

def get_data():
    db = connect_db('192.168.20.118', 'grogu', 'grogu')
    data = exe(db, 'SELECT * FROM RaspberryPi.Capteurs', verbose = True, name = True)

    return(data)

def update_plot(frame):
    data = get_data()

    df = pd.DataFrame(data[1:], columns = data[0])
    df['Date'] = df['Date'] + pd.Timedelta(hours=1)

    for ax in axes.flatten():
        ax.clear()
    for i, column in enumerate(df.columns[1:]):
        if i < 2:
            axes[0][i].plot(df['Date'], df[column], drawstyle="steps-mid")
            axes[0][i].set_title(column)
            axes[0][i].tick_params(labelrotation=45)
        else:
            axes[1][i - 2].plot(df['Date'], df[column], drawstyle="steps-mid")
            axes[1][i - 2].set_title(column)
            axes[1][i - 2].tick_params(labelrotation=45)


fig, axes = plt.subplots(2, 2, figsize=(15, 15))
plt.tight_layout()

ani = animation.FuncAnimation(fig, update_plot, get_data, interval = 300000, blit=True)

plt.show()